

# Generating Cluster Graphs

This example shows how to find the communities in a graph, then contract each community into a single node using :class:`igraph.clustering.VertexClustering`. For this tutorial, we'll use the *Donald Knuth's Les Miserables Network*, which shows the coapperances of characters in the novel *Les Miserables*.


In [101]:
import pandas as pd
import numpy as np
import tmap as tm
from mhfp.encoder import MHFPEncoder
from mhfp.lsh_forest import LSHForestHelper

df = pd.read_csv('dataset.csv')

perm = 512 
enc = MHFPEncoder(perm) 
fingerprints = [tm.VectorUint(enc.encode(s)) for s in df['canonical_smiles'].unique()]

lsh_forest = tm.LSHForest(d=128, l=8, store=True, file_backed=False, weighted=False)
lsh_forest.batch_add(fingerprints)
lsh_forest.index()

# Get the KNN graph
from_nodes = tm.VectorUint()
to_nodes = tm.VectorUint()
weights = tm.VectorFloat()
lsh_forest.get_knn_graph(from_nodes, to_nodes, weights, k=3, kc=3)

# Convert to numpy arrays if needed
from_array = np.array(from_nodes)
to_array = np.array(to_nodes)
weight_array = np.array(weights)

/home/afloresep/miniforge3/envs/tmap/lib/python3.7/site-packages/mhfp/encoder.py:297: UserWarning: The length of the shingling is 0, which results in an empty set and an all zero folded fingerprint.
  warnings.warn("The length of the shingling is 0, which results in an empty set and an all zero folded fingerprint.")


In [102]:
# Convert to numpy arrays if needed
knng_from = np.array(from_nodes)
knng_to = np.array(to_nodes)
knng_weight = np.array(weights)

In [98]:

# Assuming knng_weight is your numpy array of weights
def transform_weights(weights):
    # Shift and scale weights from [-1, 1] to (0, 1]
    transformed_weights = (weights + 1) / 2
    
    # Ensure no zeros by adding a small epsilon
    epsilon = 1e-6
    transformed_weights = np.clip(transformed_weights, epsilon, 1)
    
    return transformed_weights

# Transform the weights
weights = transform_weights(knng_weight)


In [ ]:
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt

def create_graph_and_detect_communities(from_nodes, to_nodes, weights):
    edges = list(zip(from_nodes, to_nodes))
    unique_nodes = set(from_nodes) | set(to_nodes)
    node_mapping = {node: idx for idx, node in enumerate(unique_nodes)}
    
    g = ig.Graph()
    g.add_vertices(len(unique_nodes))
    g.add_edges([(node_mapping[edge[0]], node_mapping[edge[1]]) for edge in edges])
    g.es['weight'] = weights

    # Perform Leiden community detection (still using weights)
    communities = g.community_leiden(
        objective_function='CPM',
        weights='weight',
        resolution_parameter=0.1,
        beta=0.01,
        n_iterations=50
    )

    return g, communities

def plot_graph(g, filename, communities=None):
    # Compute layout without weights
    layout = g.layout_fruchterman_reingold(
        maxiter=1000,
        area=n_vertices**2, 
        repulsedrad=
    )
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot edges (without considering weights)
    edge_xs, edge_ys = [], []
    for edge in g.es:
        x1, y1 = layout[edge.source]
        x2, y2 = layout[edge.target]
        edge_xs.extend([x1, x2, None])
        edge_ys.extend([y1, y2, None])
    ax.plot(edge_xs, edge_ys, color='lightgray', alpha=0.5, zorder=1)
    
    # Plot nodes
    if communities:
        num_communities = len(set(communities.membership))
        color_map = plt.cm.get_cmap('tab20')
        colors = [color_map(i / num_communities) for i in communities.membership]
    else:
        colors = 'skyblue'
    
    ax.scatter([coord[0] for coord in layout], [coord[1] for coord in layout], 
               c=colors, s=20, zorder=2)
    
    ax.axis('off')
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()

# Create graph and detect communities
graph, communities = create_graph_and_detect_communities(knng_from, knng_to, weights)

# Plot original graph
plot_graph(graph, "original_graph.png")

# Plot graph with communities
plot_graph(graph, "graph_with_communities.png", communities)

print("Original graph saved as 'original_graph.png'")
print("Graph with communities saved as 'graph_with_communities.png'")

# Print some basic information
print(f"Number of communities: {len(communities)}")
print(f"Modularity: {communities.modularity}")

# Save the graph with community information
graph.vs['community'] = communities.membership
graph.write_graphml("graph_with_communities.graphml")

print("Graph with communities saved as 'graph_with_communities.graphml'")

We begin by load the graph from file. The file containing this network can be
downloaded [here](http://www-personal.umich.edu/~mejn/netdata/).



In [105]:
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt

def transform_weights(weights):
    # Shift and scale weights from [-1, 1] to (0, 1]
    transformed_weights = (weights + 1) / 2
    
    # Ensure no zeros by adding a small epsilon
    epsilon = 1e-6
    transformed_weights = np.clip(transformed_weights, epsilon, 1)
    
    return transformed_weights

def create_graph_and_detect_communities(from_nodes, to_nodes, weights):
    edges = list(zip(from_nodes, to_nodes))
    unique_nodes = set(from_nodes) | set(to_nodes)
    node_mapping = {node: idx for idx, node in enumerate(unique_nodes)}
    
    g = ig.Graph()
    g.add_vertices(len(unique_nodes))
    g.add_edges([(node_mapping[edge[0]], node_mapping[edge[1]]) for edge in edges])
    g.es['weight'] = weights

    # Perform Leiden community detection
    communities = g.community_leiden(
        objective_function='CPM',
        weights='weight',
        resolution_parameter=2.0,  # Increased to create more communities
        beta=0.01,
        n_iterations=2
    )

    return g, communities, node_mapping

def plot_graph(g, filename, communities=None):
    n_vertices = g.vcount()
    
    # Compute layout with simplified parameters
    layout = g.layout_fruchterman_reingold(
        weights=g.es['weight'],
        niter=500  # Increase iterations for better layout
    )
    
    # Manually spread out the layout
    layout = np.array(layout)
    layout *= np.sqrt(n_vertices)  # Scale the layout based on number of vertices
    
    fig, ax = plt.subplots(figsize=(20, 16))  # Increased figure size
    
    # Plot edges (with reduced alpha for less visual clutter)
    edge_xs, edge_ys = [], []
    for edge in g.es:
        x1, y1 = layout[edge.source]
        x2, y2 = layout[edge.target]
        edge_xs.extend([x1, x2, None])
        edge_ys.extend([y1, y2, None])
    ax.plot(edge_xs, edge_ys, color='lightgray', alpha=0.1, zorder=1)
    
    # Plot nodes
    if communities:
        num_communities = len(set(communities.membership))
        color_map = plt.cm.get_cmap('tab20')
        colors = [color_map(i / num_communities) for i in communities.membership]
    else:
        colors = 'skyblue'
    
    # Adjust node sizes based on degree
    node_sizes = [v.degree() for v in g.vs]
    max_size = 50  # Increased max size
    node_sizes = [5 + (size/max(node_sizes)*max_size) for size in node_sizes]
    
    ax.scatter(layout[:, 0], layout[:, 1], 
               c=colors, s=node_sizes, alpha=0.7, zorder=2)
    
    ax.axis('off')
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()


# Transform weights
weights = transform_weights(knng_weight)

# Create graph and detect communities
graph, communities, node_mapping = create_graph_and_detect_communities(knng_from, knng_to, weights)

# Plot graph with communities
plot_graph(graph, "graph_with_communities.png", communities)

print("Graph with communities saved as 'graph_with_communities.png'")

# Print some basic information
print(f"Number of communities: {len(communities)}")
print(f"Modularity: {communities.modularity}")

# Save the graph with community information
graph.vs['community'] = communities.membership
graph.write_graphml("graph_with_communities.graphml")

print("Graph with communities saved as 'graph_with_communities.graphml'")

# Optional: Print community sizes
community_sizes = communities.sizes()
print("Community sizes:")
for i, size in enumerate(community_sizes):
    print(f"Community {i}: {size} nodes")

Graph with communities saved as 'graph_with_communities.png'
Number of communities: 25856
Modularity: -4.494318514211808e-05
Graph with communities saved as 'graph_with_communities.graphml'
Community sizes:
Community 0: 1 nodes
Community 1: 1 nodes
Community 2: 1 nodes
Community 3: 1 nodes
Community 4: 1 nodes
Community 5: 1 nodes
Community 6: 1 nodes
Community 7: 1 nodes
Community 8: 1 nodes
Community 9: 1 nodes
Community 10: 1 nodes
Community 11: 1 nodes
Community 12: 1 nodes
Community 13: 1 nodes
Community 14: 1 nodes
Community 15: 1 nodes
Community 16: 1 nodes
Community 17: 1 nodes
Community 18: 1 nodes
Community 19: 1 nodes
Community 20: 1 nodes
Community 21: 1 nodes
Community 22: 1 nodes
Community 23: 1 nodes
Community 24: 1 nodes
Community 25: 1 nodes
Community 26: 1 nodes
Community 27: 1 nodes
Community 28: 1 nodes
Community 29: 1 nodes
Community 30: 1 nodes
Community 31: 1 nodes
Community 32: 1 nodes
Community 33: 1 nodes
Community 34: 1 nodes
Community 35: 1 nodes
Community 3

Now that we have a graph in memory, we can generate communities using
:meth:`igraph.Graph.community_edge_betweenness` to separate out vertices into
clusters. (For a more focused tutorial on just visualising communities, check
out `tutorials-visualize-communities`).



In [64]:
communities = g.community_edge_betweenness()
communities = communities.as_clustering()

We can also easily print out who belongs to each community:



In [65]:
num_communities = len(communities)
palette = ig.RainbowPalette(n=num_communities)
for i, community in enumerate(communities):
    g.vs[community]["color"] = i
    community_edges = g.es.select(_within=community)
    community_edges["color"] = i

In [ ]:
fig, ax = plt.subplots()
ig.plot(
    communities,
    palette=palette,
    edge_width=1,
    target=ax,
    vertex_size=20,
)

# Create a custom color legend
legend_handles = []
for i in range(num_communities):
    handle = ax.scatter(
        [], [],
        s=100,
        facecolor=palette.get(i),
        edgecolor="k",
        label=i,
    )
    legend_handles.append(handle)
ax.legend(
    handles=legend_handles,
    title='Community:',
    bbox_to_anchor=(0, 1.0),
    bbox_transform=ax.transAxes,
)
plt.show()

Finally we can proceed to plotting the graph. In order to make each community
stand out, we set "community colors" using an igraph palette:



In [41]:
num_communities = len(communities)
palette1 = ig.RainbowPalette(n=num_communities)
for i, community in enumerate(communities):
    g.vs[community]["color"] = i
    community_edges = g.es.select(_within=community)
    community_edges["color"] = i

We can use a dirty hack to move the labels below the vertices ;-)



In [42]:
g.vs["label"] = ["\n\n" + label for label in g.vs["label"]]

Finally, we can plot the communities:



In [ ]:
fig1, ax1 = plt.subplots()
ig.plot(
    communities,
    target=ax1,
    mark_groups=True,
    palette=palette1,
    vertex_size=15,
    edge_width=0.5,
)
fig1.set_size_inches(20, 20)

Now let's try and contract the information down, using only a single vertex
to represent each community. We start by defining x, y, and size attributes
for each node in the original graph:



In [44]:
layout = g.layout_kamada_kawai()
g.vs["x"], g.vs["y"] = list(zip(*layout))
g.vs["size"] = 15
g.es["size"] = 15

Then we can generate the cluster graph that compresses each community into a
single, "composite" vertex using
:meth:`igraph.VertexClustering.cluster_graph`:



In [45]:
cluster_graph = communities.cluster_graph(
    combine_vertices={
        "x": "mean",
        "y": "mean",
        "color": "first",
        "size": "sum",
    },
    combine_edges={
        "size": "sum",
    },
)

<div class="alert alert-info"><h4>Note</h4><p>We took the mean of x and y values so that the nodes in the cluster
     graph are placed at the centroid of the original cluster.</p></div>

<div class="alert alert-info"><h4>Note</h4><p>``mean``, ``first``, and ``sum`` are all built-in collapsing functions,
    along with ``prod``, ``median``, ``max``, ``min``, ``last``, ``random``.
    You can also define your own custom collapsing functions, which take in a
    list and return a single element representing the combined attribute
    value. For more details on igraph contraction, see
    :meth:`igraph.GraphBase.contract_vertices`.</p></div>



Finally, we can assign colors to the clusters and plot the cluster graph,
including a legend to make things clear:



In [ ]:
palette2 = ig.GradientPalette("gainsboro", "black")
g.es["color"] = [palette2.get(int(i)) for i in ig.rescale(cluster_graph.es["size"], (0, 255), clamp=True)]

fig2, ax2 = plt.subplots()
ig.plot(
    cluster_graph,
    target=ax2,
    palette=palette1,
    # set a minimum size on vertex_size, otherwise vertices are too small
    vertex_size=[max(20, size) for size in cluster_graph.vs["size"]],
    edge_color=g.es["color"],
    edge_width=0.8,
)

# Add a legend
legend_handles = []
for i in range(num_communities):
    handle = ax2.scatter(
        [], [],
        s=100,
        facecolor=palette1.get(i),
        edgecolor="k",
        label=i,
    )
    legend_handles.append(handle)

ax2.legend(
    handles=legend_handles,
    title='Community:',
    bbox_to_anchor=(0, 1.0),
    bbox_transform=ax2.transAxes,
)

fig2.set_size_inches(10, 10)